In [1]:
!pip install \
transformers \
datasets \
transformers[torch] \
transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 51.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.7 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

In [3]:
dataset = load_dataset("cnn_dailymail", version="3.0.0")
print(f"Features: {dataset['train'].column_names}")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Features: ['article', 'highlights', 'id']


In [4]:
sample = dataset['train'][1]
print(f"""
Article (excerpt of 500 characters, total length: {len(sample["article"])}):
      """)
print(sample["article"][:500])
print(f"\nSummary (length: {len(sample['highlights'])}):")
print(sample["highlights"])


Article (excerpt of 500 characters, total length: 4051):
      
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events. Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial. MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor." Here, inmates with the most s

Summary (length: 281):
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .


# Text Summarization Pipeline

In [5]:
import nltk
from nltk.tokenize import sent_tokenize

In [6]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
string = "Mr. Ronit is paying $10. The shopkeeper is selling that item."
sent_tokenize(string)

['Mr. Ronit is paying $10.', 'The shopkeeper is selling that item.']

In [8]:
sample_text = dataset["train"][1]["article"][:2000]

In [9]:
summaries = {}

In [10]:
def three_sentence_summary(text):
  return "\n".join(sent_tokenize(text)[:3])

summaries["baseline"] = three_sentence_summary(sample_text)
summaries

{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."'}

# GPT 2

In [11]:
from transformers import pipeline, set_seed
set_seed(42)
pipe = pipeline("text-generation", model="gpt2-xl")
gpt2_query = sample_text + "\nTL:DR:\n"
pipe_out = pipe(gpt2_query, max_length=512, clean_up_tokenization_spaces=True)
summaries["gpt2"] = "\n".join(
    sent_tokenize(pipe_out[0]["generated_text"][len(gpt2_query):])
)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [12]:
summaries

{'baseline': 'Editor\'s note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.\nHere, Soledad O\'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.\nMIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."',
 'gpt2': "I'm not an expert on mental illness and would be happy to learn of an expert who can vouch for this place.\nA mental illness is not like a physical illness because, if you have a mental illness, you usually don't know you have a mental illness.\nThere is no cure for a mental illness.\nThere is also no cure for becoming a criminal.\nFor the majority"}

# T5

In [13]:
pipe = pipeline("summarization", model="t5-large")
pipe_out = pipe(sample_text)
summaries["t5"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-large automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [14]:
print(summaries["t5"])

mentally ill inmates are housed on the ninth floor of a florida jail .
most face drug charges or charges of assaulting an officer .
judge says arrests often result from confrontations with police .
one-third of all people in Miami-dade county jails are mental ill .


# BART

In [15]:
pipe = pipeline("summarization", model="facebook/bart-large-cnn")
pipe_out = pipe(sample_text)
summaries["bart"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"]))

In [16]:
print(summaries["bart"])

Mentally ill inmates are housed on the "forgotten floor" of Miami-Dade jail.
Most often, they face drug charges or charges of assaulting an officer.
Judge Steven Leifman says the arrests often result from confrontations with police.
He says about one-third of all people in the county jails are mentally ill.


# PEGASUS

In [17]:
pipe = pipeline("summarization", model="google/pegasus-cnn_dailymail")
pipe_out = pipe(sample_text)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
summaries["pegasus"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"].replace(" .<n>", ".\n").replace("<n>", ".\n")))

In [19]:
print(summaries["pegasus"])

Mentally ill inmates in Miami are housed on the "forgotten floor".
The ninth floor is where they're held until they're ready to appear in court.
Most often, they face drug charges or charges of assaulting an officer.
They end up on the ninth floor severely mentally disturbed .


# Comparing different summaries

In [20]:
print("GROUND TRUTH")
print(dataset["train"][1]["highlights"])
print("")
for model_name in summaries:
  print(model_name.upper())
  print(summaries[model_name])
  print("")

GROUND TRUTH
Mentally ill inmates in Miami are housed on the "forgotten floor"
Judge Steven Leifman says most are there as a result of "avoidable felonies"
While CNN tours facility, patient shouts: "I am the son of the president"
Leifman says the system is unjust and he's fighting for change .

BASELINE
Editor's note: In our Behind the Scenes series, CNN correspondents share their experiences in covering news and analyze the stories behind the events.
Here, Soledad O'Brien takes users inside a jail where many of the inmates are mentally ill. An inmate housed on the "forgotten floor," where many mentally ill inmates are housed in Miami before trial.
MIAMI, Florida (CNN) -- The ninth floor of the Miami-Dade pretrial detention facility is dubbed the "forgotten floor."

GPT2
I'm not an expert on mental illness and would be happy to learn of an expert who can vouch for this place.
A mental illness is not like a physical illness because, if you have a mental illness, you usually don't know y

# Evaluation

In [21]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.6 MB/s eta 0:00:00


In [22]:
from datasets import load_metric
bleu_metric = load_metric("sacrebleu")

<ipython-input-22-569b9a148574>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("sacrebleu")


In [23]:
import pandas as pd
import numpy as np
bleu_metric.add(
    predictions="hello hello hello hello hello hello", reference=["hello I am Ronit, and hello again"]
)

results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Values"])

,Values
score,0.0
counts,"[2, 0, 0, 0]"
totals,"[6, 5, 4, 3]"
precisions,"[33.33, 0.0, 0.0, 0.0]"
bp,0.716531
sys_len,6
ref_len,8


In [24]:
bleu_metric.add(
    predictions="hello I Ronit, and hello", reference=["hello I am Ronit, and hello again"]
)

results = bleu_metric.compute(smooth_method="floor", smooth_value=0)
results["precisions"] = [np.round(p, 2) for p in results["precisions"]]
pd.DataFrame.from_dict(results, orient="index", columns=["Values"])

,Values
score,43.298201
counts,"[6, 4, 2, 1]"
totals,"[6, 5, 4, 3]"
precisions,"[100.0, 80.0, 50.0, 33.33]"
bp,0.716531
sys_len,6
ref_len,8


In [25]:
!pip install rouge_score
rouge_metric = load_metric("rouge")

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=ef3a6c875c68026c29973fe71bd94bbdb71027c86f406d84f70a2b59bf662c30
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [26]:
reference = dataset["train"][1]["highlights"]
records = []
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]

In [27]:
for model_name in summaries:
  rouge_metric.add(prediction=summaries[model_name], reference=reference)
  score = rouge_metric.compute()
  rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
  records.append(rouge_dict)


pd.DataFrame.from_records(records, index=summaries.keys())

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.365079,0.145161,0.206349,0.285714
gpt2,0.152542,0.000000,0.084746,0.152542
t5,0.382979,0.130435,0.255319,0.382979
bart,0.475248,0.222222,0.316832,0.415842
pegasus,0.326531,0.208333,0.285714,0.326531


# Evaluating pegasus on the CNN/DailyMail Dataset

In [28]:
def evaluate_summaries_baseline(dataset, metric,
                                column_text="article", column_summary="highlights"):
  summaries = [three_sentence_summary(text) for text in dataset[column_text]]
  metric.add_batch(predictions=summaries,
                   references=dataset[column_summary])

  score = metric.compute()
  return score

In [29]:
test_sampled = dataset["test"].shuffle(seed=42).select(range(1000))

In [30]:
score = evaluate_summaries_baseline(test_sampled, rouge_metric)
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame.from_dict(rouge_dict, orient="index", columns=["baseline"]).T

,rouge1,rouge2,rougeL,rougeLsum
baseline,0.389276,0.171296,0.245061,0.354239


In [31]:
from tqdm import tqdm
import torch

In [32]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [33]:
def chunks(list_of_elements, batch_size):
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i: i+batch_size]

In [34]:
def evaluate_summaries_pegasus(dataset, metric, model, tokenizer,
                               batch_size=16, device=device, column_text="article", column_summary="highlights"):
  article_batches=list(chunks(dataset[column_text], batch_size))
  target_batches = list(chunks(dataset[column_summary], batch_size))

  for article_batches, target_batches in tqdm(
      zip(article_batches, target_batches), total=len(article_batches)
  ):
    inputs = tokenizer(article_batches, max_length=1024, truncation=True, padding="max_length", return_tensors="pt")
    summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                   attention_mask=inputs["attention_mask"].to(device),
                   length_penalty=0.8,
                   num_beams=8,
                   max_length=128)

    decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True,
                                          clean_up_tokenization_spaces=True)
    for s in summaries]

    decoded_summaries = [d.replace("<n>", " ") for d in decoded_summaries]
    metric.add_batch(predictions=decoded_summaries, references=target_batches)

    score = metric.compute()
    return score

In [35]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [36]:
model_ckpt = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [37]:
score = evaluate_summaries_pegasus(test_sampled, rouge_metric, model, tokenizer, batch_size=8)
score

  0%|          | 0/125 [00:07<?, ?it/s]


{'rouge1': AggregateScore(low=Score(precision=0.2514869879850981, recall=0.33116296993427513, fmeasure=0.28624853254090543), mid=Score(precision=0.3437029872900844, recall=0.4737559764990129, fmeasure=0.389936401488888), high=Score(precision=0.4096958605869597, recall=0.5893576269297783, fmeasure=0.4648925136790579)),
 'rouge2': AggregateScore(low=Score(precision=0.0699415672065928, recall=0.09125814646479324, fmeasure=0.07927820020543086), mid=Score(precision=0.13225348110258595, recall=0.18464811794553174, fmeasure=0.14982892254522812), high=Score(precision=0.187601758915215, recall=0.2836408659299481, fmeasure=0.22076496480762817)),
 'rougeL': AggregateScore(low=Score(precision=0.16414539170788528, recall=0.22133919674150382, fmeasure=0.1842511509412843), mid=Score(precision=0.23173288160301697, recall=0.3242664500382681, fmeasure=0.2632261968176638), high=Score(precision=0.2949539511157585, recall=0.4276349041388396, fmeasure=0.33580665785799013)),
 'rougeLsum': AggregateScore(low=

In [38]:
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.389936,0.149829,0.263226,0.321016


# Training on SAMSum Dataset

In [39]:
!pip install py7zr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 43.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.8/49.8 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 14.4 MB/s eta 0:00:00


In [40]:
samsum_dataset = load_dataset("samsum")

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [41]:
samsum_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [42]:
split_length = [len(samsum_dataset[split]) for split in samsum_dataset]
split_length

[14732, 819, 818]

In [43]:
print(f"Split lengths: {split_length}")
print(f"Features: {samsum_dataset['train'].column_names}")
print("\nDialogue:")
print(samsum_dataset['test'][0]['dialogue'])
print("\nSummary")
print(samsum_dataset['test'][0]['summary'])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Summary
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.


## Evaluating Pegasus on samsum

In [44]:
pipe_out = pipe(samsum_dataset['test'][0]["dialogue"])
print("Summary:")
pipe_out

Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Summary:


[{'summary_text': "Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him ."}]

In [45]:
print(pipe_out[0]["summary_text"].replace(" .<n>", ".\n"))

Amanda: Ask Larry Amanda: He called her last time we were at the park together.
Hannah: I'd rather you texted him.
Amanda: Just text him .


In [46]:
score = evaluate_summaries_pegasus(samsum_dataset['test'], rouge_metric, model,
                                   tokenizer, column_text="dialogue",
                                   column_summary="summary", batch_size=8)
print(score)

  0%|          | 0/103 [00:03<?, ?it/s]

{'rouge1': AggregateScore(low=Score(precision=0.14484363696648184, recall=0.286075810913497, fmeasure=0.1948646737227217), mid=Score(precision=0.24922655850247316, recall=0.4002186257015611, fmeasure=0.26711692217605854), high=Score(precision=0.3788046410919295, recall=0.5070377564755585, fmeasure=0.356215031553085)), 'rouge2': AggregateScore(low=Score(precision=0.015306122448979591, recall=0.03362068965517241, fmeasure=0.018748548873926164), mid=Score(precision=0.0647020456928291, recall=0.08213259441707718, fmeasure=0.06050691911363271), high=Score(precision=0.14443517527978933, recall=0.14226934523809523, fmeasure=0.11255367346879358)), 'rougeL': AggregateScore(low=Score(precision=0.10722085744715056, recall=0.21200678228625902, fmeasure=0.1450660721296641), mid=Score(precision=0.17752725179922407, recall=0.30068378668367324, fmeasure=0.19658113507089814), high=Score(precision=0.2727156132739212, recall=0.39525403324745434, fmeasure=0.263831578217267)), 'rougeLsum': AggregateScore(l

In [47]:
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.267117,0.060507,0.196581,0.196891


# Fine-Tuning Pegasus

In [48]:
def convert_examples_to_features(example_batch):
  input_encodings = tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)
  with tokenizer.as_target_tokenizer():
    target_encodings=tokenizer(example_batch["summary"], max_length=128, truncation=True)

  return {"input_ids": input_encodings["input_ids"],
          "attention_mask": input_encodings["attention_mask"],
          "labels": target_encodings["input_ids"]}

In [49]:
dataset_samsum_pt = samsum_dataset.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3635: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [50]:
columns = ["input_ids", "labels", "attention_mask"]
dataset_samsum_pt.set_format(type='torch', columns=columns)
dataset_samsum_pt

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [51]:
from transformers import DataCollatorForSeq2Seq

In [52]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [53]:
from transformers import Trainer, TrainingArguments

In [54]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)

In [55]:
training_args = TrainingArguments(
    output_dir='fine-tuned-pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1,
    weight_decay=0.1, logging_steps=10, push_to_hub=True,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

In [56]:
from huggingface_hub import notebook_login
notebook_login()

In [58]:
trainer = Trainer(model=model, args=training_args,
                  tokenizer=tokenizer,
                  data_collator=seq2seq_data_collator,
                  train_dataset=dataset_samsum_pt["train"],
                  eval_dataset=dataset_samsum_pt["validation"])

Cloning https://huggingface.co/ronit33/fine-tuned-pegasus-samsum into local empty directory.


In [59]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.664700,1.482927


TrainOutput(global_step=920, training_loss=1.8359185861504597, metrics={'train_runtime': 1812.1913, 'train_samples_per_second': 8.129, 'train_steps_per_second': 0.508, 'total_flos': 5526698901602304.0, 'train_loss': 1.8359185861504597, 'epoch': 1.0})

In [60]:
score = evaluate_summaries_pegasus(
    samsum_dataset["test"], rouge_metric, trainer.model, tokenizer,
    batch_size=2, column_text="dialogue", column_summary="summary"
)
print(score)

  0%|          | 0/410 [00:01<?, ?it/s]

{'rouge1': AggregateScore(low=Score(precision=0.32142857142857145, recall=0.5625, fmeasure=0.40909090909090917), mid=Score(precision=0.3807142857142857, recall=0.7395833333333333, fmeasure=0.5018427518427518), high=Score(precision=0.44, recall=0.9166666666666666, fmeasure=0.5945945945945945)), 'rouge2': AggregateScore(low=Score(precision=0.07407407407407407, recall=0.13333333333333333, fmeasure=0.09523809523809523), mid=Score(precision=0.2037037037037037, recall=0.4303030303030303, fmeasure=0.2761904761904762), high=Score(precision=0.3333333333333333, recall=0.7272727272727273, fmeasure=0.4571428571428572)), 'rougeL': AggregateScore(low=Score(precision=0.21428571428571427, recall=0.375, fmeasure=0.2727272727272727), mid=Score(precision=0.30714285714285716, recall=0.6041666666666667, fmeasure=0.40663390663390664), high=Score(precision=0.4, recall=0.8333333333333334, fmeasure=0.5405405405405406)), 'rougeLsum': AggregateScore(low=Score(precision=0.21428571428571427, recall=0.375, fmeasure

In [61]:
rouge_dict = dict((rn, score[rn].mid.fmeasure) for rn in rouge_names)
pd.DataFrame(rouge_dict, index=["pegasus"])

,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.501843,0.27619,0.406634,0.406634


In [62]:
trainer.push_to_hub(commit_message="Trained Successfully")

Upload file pytorch_model.bin:   0%|          | 1.00/2.13G [00:00<?, ?B/s]

Upload file runs/Aug19_10-33-15_7de7ac806a64/events.out.tfevents.1692441238.7de7ac806a64.1446.0:   0%|        …

Upload file spiece.model:   0%|          | 1.00/1.82M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.87k [00:00<?, ?B/s]

To https://huggingface.co/ronit33/fine-tuned-pegasus-samsum
   c6e5341..92bc651  main -> main

   c6e5341..92bc651  main -> main

To https://huggingface.co/ronit33/fine-tuned-pegasus-samsum
   92bc651..149be8c  main -> main

   92bc651..149be8c  main -> main



'https://huggingface.co/ronit33/fine-tuned-pegasus-samsum/commit/92bc651a49f6d8a6971c66327513263d3d0a1a78'

In [64]:
from transformers import pipeline
def summarize_text(text):
  model_id = "ronit33/fine-tuned-pegasus-samsum"
  pipe = pipeline("summarization", model=model_id)

  pipe_out = pipe(text)
  summaries = {}
  summaries["summary"] = "\n".join(sent_tokenize(pipe_out[0]["summary_text"].replace(" .<n>", ".\n").replace("<n>", ".\n")))

  return summaries


In [70]:
sample_text = samsum_dataset['test'][10]
print(sample_text['dialogue'])

Wanda: Let's make a party!
Gina: Why?
Wanda: beacuse. I want some fun!
Gina: ok, what do u need?
Wanda: 1st I need too make a list
Gina: noted and then?
Wanda: well, could u take yours father car and go do groceries with me?
Gina: don't know if he'll agree
Wanda: I know, but u can ask :)
Gina: I'll try but theres no promisess
Wanda: I know, u r the best!
Gina: When u wanna go
Wanda: Friday?
Gina: ok, I'll ask


In [71]:
print(summarize_text(sample_text['dialogue'])['summary'])

Wanda wants to make a party on Friday.
Gina will take Wanda's father car and go do groceries with her.
Gina will ask Wanda when she wants to go.


In [73]:
sample_text['summary']

"Wanda wants to throw a party. She asks Gina to borrow her father's car and go do groceries together. They set the date for Friday. "